<a href="https://colab.research.google.com/github/vkgithub1556/ReliabilityProject/blob/main/Transformers_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import math
import numpy as np
from scipy.special import gamma


In [2]:
data = pd.read_csv('/content/drive/MyDrive/Data/transformers/subset_ensemble_BCCAQv2_53.532_-113.516_365_day.csv')

In [3]:
data = data.groupby('time').mean().reset_index()
data['time'] =  pd.to_datetime(data['time'])
data['Year'] = data['time'].dt.year

In [4]:
canESM2 = pd.DataFrame()
canESM2 = data.filter(['time','Year','tasmax_CanESM2_historical_rcp85_(degC)',
       'tasmax_CanESM2_historical_rcp45_(degC)',
       'tasmax_CanESM2_historical_rcp26_(degC)'],axis=1)

In [5]:
canESM2_85 = pd.DataFrame(data=canESM2.filter(['time','Year','tasmax_CanESM2_historical_rcp85_(degC)']))
canESM2_45 = pd.DataFrame(data=canESM2.filter(['time','Year','tasmax_CanESM2_historical_rcp45_(degC)']))
canESM2_26 = pd.DataFrame(data=canESM2.filter(['time','Year','tasmax_CanESM2_historical_rcp26_(degC)']))

In [6]:
canESM2_85 = canESM2_85[(canESM2_85['tasmax_CanESM2_historical_rcp85_(degC)']>25) & (canESM2_85['Year'] > 2020)].reset_index(drop=True)
canESM2_45 = canESM2_45[(canESM2_45['tasmax_CanESM2_historical_rcp45_(degC)']>25) & (canESM2_45['Year'] > 2020)].reset_index(drop=True)
canESM2_26 = canESM2_26[(canESM2_26['tasmax_CanESM2_historical_rcp26_(degC)']>25) & (canESM2_26['Year'] > 2020)].reset_index(drop=True)

In [7]:
def life_expectancy(df,temperature):
 df['FAA'] = np.exp((15000/383)-(15000/(df[temperature]+65+273)))
 df['FAA_Cummulative'] = df.groupby('Year').cumsum()['FAA']

 LOL = df.groupby('Year').tail(1).reset_index(drop=True)[['Year','FAA_Cummulative']]

 LOL['Days_gt_25degC'] = df.groupby('Year').count().reset_index()[temperature]
 LOL['LossOfLife_Percentage'] = (((LOL['FAA_Cummulative']/LOL['Days_gt_25degC'])*100)/20.55)
 LOL['LossOfLife_Percentage_wrt2021'] = LOL.LossOfLife_Percentage[0] - LOL['LossOfLife_Percentage']
 return(LOL)

In [8]:
life_expectancy(canESM2_85,'tasmax_CanESM2_historical_rcp85_(degC)')

,Year,FAA_Cummulative,Days_gt_25degC,LossOfLife_Percentage,LossOfLife_Percentage_wrt2021
0,2021,3.848982,28,0.668923,0.000000
1,2022,12.028137,73,0.801796,-0.132873
2,2023,12.295067,69,0.867102,-0.198179
3,2024,9.769152,59,0.805736,-0.136814
4,2025,11.797735,65,0.883229,-0.214306
...,...,...,...,...,...
75,2096,38.318558,126,1.479881,-0.810958
76,2097,30.292487,113,1.304502,-0.635579
77,2098,37.845486,126,1.461611,-0.792688
78,2099,45.336633,127,1.737136,-1.068213


In [9]:
canESM2_85.groupby('Year').tail(1).reset_index(drop=True)[['Year','FAA_Cummulative']]

,Year,FAA_Cummulative
0,2021,3.848982
1,2022,12.028137
2,2023,12.295067
3,2024,9.769152
4,2025,11.797735
...,...,...
75,2096,38.318558
76,2097,30.292487
77,2098,37.845486
78,2099,45.336633
